#### Física de la Radioterapia
# Visualización de tratamientos de radioterapia
## Distribuciones de dosis absorbida
-----
*Este cuadereno modifica el código de los cuadernos de ejemplo del módulo [scikit-rt](https://scikit-rt.github.io/scikit-rt/examples.html)*

Los datos corresponden a una paciente real tratada en el Hospital de Fuenlabrada.

Han sido anonimizados de acuerdo a procedimientos validados.

**Objetivos**: 

* Mostrar elementos que aparecen en la planificación de un tratamiento de radioterapia

* Comprender las bases radiobiológicas del fraccionamiento de la dosis en radioterapia

## Fundamentos

Los conceptos básicos sobre los que se trabajará en este cuaderno corresponden a un tratamiento de mama, una de las patalogías que con más frecuenca se trata en los Servicios de Radioterapia, pero son aplicables en general a cualquier otra patología. 

En el tratamiento del cáncer de mama la radioterapia se emplea como tratamiento adyuvante a la cirugía. El tumor primario se extirpa y posteriormente se administra una dosis de radiación sobre el lecho tumoral y sobre el resto de la mama.

Definiendo un volumen de tratamiento que comprenda el lecho tumoral más un margen para incluir la posible enfermedad microscópica, la experiencia clínica apunta a que se tiene una alta probabilidad de éxito en el tratamiento si la dosis que se administra a ese volumen es de 60 Gy en 30 fracciones. Sobre el volumen que cubre toda la mama la dosis administrada debe ser de 50 Gy en 25 fracciones.

Los efectos adversos del tratamiento se cuantifican fundamentalmente por la dosis que reciba el pulmón ipsilateral, siendo un buen indicador predictivo el porcentaje de volumen que recibe una dosis de 20 Gy o superior en relación al volumen total de ese pulmón. Se considera tolerable un tratamiento en el que el volumen de pulmón que recibe 20 Gy es inferior al 25\% del volumen total. Otro efecto adverso es la aparición de radiodermitis por respuesta aguda de la piel a la radiación. Este efecto es controlado directamente por el oncólogo de radioterapia mediante revisiones en consulta durante el curso del tratamiento. En casos graves puede complicar el desarrollo del tratamiento obligando a parar y descansar durante un tiempo.

La manera convencional de aplicar el tratamiento es hacerlo de manera secuencial. Durante 25 fracciones se irradia la mama completa para generar una distribución de dosis todo lo uniforme posible procurando evitar el pulmón. El valor medio de esa distribución de dosis será de 50 Gy y la dispersión de la dosis debe ser tal que no más de un 5\% de la mama reciba más de 53.5 Gy ni más de un 5\% de la mama reciba menos de un 47.5 Gy, o dicho en términos relativos, el 95\% del volumen de la mama recibe al menos el 95\% de la dosis de prescripción y no más del 5\% del volumen de la mama recibe más del 107\% de la dosis de prescripción. A continuación se administraran otras cinco fracciones en las se procurará irradiar únicamente el volumen del lecho tumoral hasta que en este volumen se tenga una distribución de dosis con un valor medio de 60 Gy y de nuevo con una dispersión tal que se cumplan los criterios del 5\% aplicados al 95\% y al 107\% de la dosis de prescripción. En el argot de la radioterapia a la irradiación del volumen reducido se la denomina \textit{Boost} o Sobreimpresión.

Los avances técnicos han permitido realizar el tratamiento de manera concurrente. Los dos volúmenes se irradian en el mismo número de sesiones prescribiendo diferente dosis por sesión en cada uno de ellos. El tratamiento se acorta en el tiempo y la distribución de dosis se optimiza en mayor medida. La tendencia actual es realizar los tratamientos en menos fracciones.

En radioterapia la evaluación de los planes de tratamiento se realiza mediante mapas de dosis sobre cortes tomográficos e histogramas dosis volumen (DVH). Los oncólogos de radioterapia valoran la conformación de la dosis a los volúmenes de tratamiento y la protección de las estructuras de riesgo. Los DVH reducen la información de la distribución de dosis permitiendo evaluar la cantidad de volumen irradiado con una dosis dada. Lo habitual es presentarlos como histogramas acumulados invertidos de manera que cada punto del histograma nos da el volumen que recibe al menos la dosis considerada.

## Visualización de distribuciones de dosis

Esta primera parte del cuaderno emplea el módulo python `scikit-rt` para generar gráficos interactivos que permiten visualizar distribuciones de dosis empleadas en radioterapia.

Todos los sistemas de planificación de radioterapia ofrecen estas mismas representaciones. Los gráficos presentados aquí son menos versátiles que los de los programas de planificación pero básicamente tienen la misma funcionalidad.

Se asume que estamos corriendo el cuaderno en `jupyterlab`, bien sea en la distribución proporcionada en `mybinder` o en una instalación local.

Comenzamos por importar una serie de módulos que nos harán falta y por configurar algunas opciones

In [ ]:
from skrt import Dose, Patient
from skrt.core import get_data_by_filename
import gdown

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

# Set scikit-rt viewer options.
options = {}
# Make display interactive ("no_ui": False) or non-interactive ("no_ui": True).
options["no_ui"] = False
# Include the interface for interactive zooming.
options["zoom_ui"] = not options["no_ui"]
# Use centre left of legend as reference point for fixing its location.
options["legend_loc"] = "center left"
# Set coordinates of legend reference point,
# in system where image axes extend from 0 to 1.
options["legend_bbox_to_anchor"] = (1.8, 0.5)
# Set figure size in inches.
options["figsize"] = (13, 6)
# Overlay default image annotation (slice z-coordinate), in white, at default position (top left or image).
options["annotate_slice"] = {"color": "white", "fontsize": 24}

### Subida de datos 

Los datos de la paciente con las imágenes de CT, estructuras anatómicas relevantes, planes de radioterapia y distribucines de dosis, se han comprimido y compartido en un archivo zip disponible en [Google Drive](https://drive.google.com/file/d/1ICECuwHhMBoMwioQ96-UpkV43v0kNneI/view?usp=sharing).

Al ejecutar el código de la siguiente celda se coloca toda la información de la paciente en la carpeta `data/MamaDer001`

In [ ]:
# Descargar los datos compartidos de Google Drive
url = 'https://drive.google.com/file/d/1ICECuwHhMBoMwioQ96-UpkV43v0kNneI/view?usp=sharing'
output = '../data/MamaDer001.zip'
gdown.download(url, output, quiet=False, fuzzy=True)

# Descomprimir el archivo zip
!bash ../shell_scripts/preparar_datos > /dev/null

Leer los datos de la paciente e identificar los planes de tratamiento y sus correspondientes distribuciones de dosis:
- *Secuencial*: compuesto por **mama** y **boost**. La combinación de ambos corresponde al plan **suma**
- *Concurrente*: plan **SIB** del inglés **S**imultaneous **I**ntegrated **B**oost

In [ ]:
patient = Patient('../data/MamaDer001', unsorted_dicom=True)
(suma, boost, SIB, mama) = (0, 1, 2, 3)

Al ejecutar la siguiente celda se generara un gráfico interactivo que nos permite visualizar las distribuciones de dosis correspondientes a los diferentes esquemas de tratamiento. 

- Para seleccionar un esquema dado basta con modificar el `plan_ID`asociado
- Para visualizar planos axiales tiene que estar seleccionado `x-y` en `Slice plane selection`
- Hacer visible la segmentación de las estructuras anatómicas relevantes. Seleccionar una opción en `ROI plotting` distinta de `none`. Lo habitual en Radioterapia es trabajar en la opción `contour`
- Hacer visible la distribución de dosis mediante el control `Dose opacity`
- Seleccionar las estructuras relevantes para lo que vamos a trabahar en este cuaderno: PTV boost dcho, PTV mama dcha, Pulmon Dcho.
- Hacer `Zoom` y centrar la imagen en las regiones que se irradian directamente mediante los controles `x centre (mm)` e `y centre (mm)`. La figura debería mostrar una imagen similar a esta
![image](../images/zoom.png)
- Desplazarse en la dirección craneo caudal cambiando el valor del control `z (mm)`. Una vez seleccionado este control los cursores del teclado nos permiten desplazarnos corte a corte del estudio de CT.

In [ ]:
plan_ID = suma # Para representar otras distribuciones de dosis cambiar el plan_ID seleccionando (mama, boost, suma, SIB)
# Obtain references to selected objects.
ct = patient.studies[0].ct_images[0]
dose = patient.studies[0].ct_doses[plan_ID]
plan = patient.studies[0].ct_plans[plan_ID]
plan.load_constraints()
ss1 = patient.studies[0].ct_structure_sets[0]
ss1.set_image(ct)
ss1.name = "plan"
dose.resample(voxel_size=ct.voxel_size)
dose.origin = ct.origin
# Set options differently depending on whether notebook is to be used interactively.
roi_plot_type = "contour" if options["no_ui"] else "none"
dose_opacity = 0.5 if options["no_ui"] else None

# Display image and associated data.
ct.view(dose=dose, rois=ss1, roi_plot_type=roi_plot_type, legend=True,
        colorbar=2, dose_opacity=dose_opacity, **options);

#### Cuestiones

1. Desplazándose por las imágenes de CT determinar cuál es la extensión en la dirección craneo caudal de
- PTV mama dcha
- PTV boost dcho
- Pulmón dcho

En las imágenes se pueden apreciar unos objetos brillantes colocados sobre la superficie de la paciente. Sirven para señalar la cicatriz resultado de la intervención quirúrgica previa a la radioterapia.

2. Determinar la extensión de la cicatriz en la dirección craneo caudal.
1. Indicar si la posición de la cicatriz en la mama coincide con el volumen de PTV boost dcho.
1. ¿Qué estructura tiene mayor extensión la cicatriz o el PTV boost dcho? Comentar qué razones pensamos que motivan esta diferencia de extensión

Dentro de la mama se puden apreciar puntos brillantes que claramente no son de origen natural. Son grapas quirúrgicas.

5. Indicar en qué cortes tomográgicos se ven grapas 
1. ¿Se encuentran todas las grapas dentro del volumen PTV boost dcho? Comentar la relación entre la posición de las grapas y la del PTV boost.
1. Cambiando en el gráfico el `plan_ID` entre `{mama, boost, suma}` para cada uno de los planes describir, de forma aproximada, la extensión de los niveles de dosis de tratamiento explicados en la sección **Fundamentos**
1. Seleccinando como `plan_ID` `SIB` realizar a partir de la figura una primera estimación de las dosis de tratamiento en el PTV mama dcha y el PTV boost dcho

## Histogramas dosis volumen

Como ya hemos indicado en **Fundamentos** visualizar la dosis permite valorar su distribución espacial pero no es un método sencillo de realizar evaluaciones cuantitativas. 

En las celdas siguientes mostraremos gráficas de *DVH*, histograma dosis volumen, para ayudar a comprender en qué consisten estos objetos.

Comenzamos introduciendo una identificación de las estructuras relevantes

In [ ]:
ptvboost, ptvmama, pulmonder = 7, 8, 9

`scikit-rt` proporciona herramientas para obtener y representar *DVH*s a partir de una distribución de dosis y un conjunto de estructuras. La siguiente celda muestra un ejemplo de cómo hacerlo para el plan `suma`

In [ ]:
plan_ID = suma
dose = patient.studies[0].ct_doses[plan_ID]
dose.plot_dvh([ss1[ptvboost], ss1[ptvmama], ss1[pulmonder]], bins=1000, figsize=(12,6));

La figura anterior muestra los *DVH*s pero no nos da valores concretos sobre los mismos. Para poder realizar una evaluadción más detallada necesitamos utilizar algunas funciones de otro modulo orientado a la radioterapia denominado `dicompylercore`

In [ ]:
from dicompylercore import dvh, dvhcalc

El objeto `dvhcalc` tiene un método denominado `get_dvh` que devuelve un objeto `dvh` correspondiente a una estructura dada dentro de un conjunto de estructuras y una distribución de dosis.

En la siguiente celda identificamos los archivos DICOM con las estructuras y las distribuciones de dosis

In [ ]:
# Archivo DICOM de estructuras: órganos blanco y órganos de riesgo
dcm_ss = '../data/MamaDer001/20230116_095704/RTSTRUCT/CT/20230116_095704/RS.1.2.246.352.205.5566667512400723974.375329800850442140.dcm'
# Archivos DICOM de distribuciones de dosis
dcm_dose_mama = '../data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.56180825880144060698193516247443136422.dcm'
dcm_dose_boost = '../data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.488893207455186615011534392364350689689.dcm'
dcm_dose_suma = '../data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.47352446189117142439988284232660965553.dcm'
dcm_dose_SIB = '../data/MamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/RD.1.2.246.352.221.52708297311437444114737711031232263301.dcm'

La siguiente celda sirve para definir los índices de las estructuras relevantes según figuran en el archivo DICOM

In [ ]:
dcm_ptvboost, dcm_ptvmama, dcm_pulmonder = 14, 15, 16

Con estas identificaciones para obtener el *DVH* correspondiente al PTV boost dcho en la distribución de dosis del plan `suma` tendremos que escribir

In [ ]:
ptvboost_dvh = dvhcalc.get_dvh(dcm_ss, dcm_dose_suma, dcm_ptvboost)

El objeto `dvh` creado tiene muchos métodos útiles para hacer evaluaciones cuantitativas. Una primera reducción de información se puede obtener mediante el método `describe`. Por ejemplo la celda siguiente muestra el resumen correspondiente a `ptv_boost` recién creado

In [ ]:
ptvboost_dvh.describe()

Dado que un *DVH* es una curva que relaciona dosis con volumen, los objetivos de prescripción y las restricciones de dosis que se imponen sobre los *DVH* pueden venir expresadas de dos maneras diferentes:

- **Dxx** nos dice la dosis que recibe un volumen **xx**. Si **xx** se expresa sin unidades se refeiere a un porcentaje de volumen respecto al volumen total del órgano. Si se da con unidades se refiere lógicamente a volumen medido en esas unidades. Las propias dosis se pueden dar en valor absoluto medido en las unidades de dosis correspondiente o en valores relativos respecto a la dosis de prescripción del tratamiento.
- **Vxx** nos dice el volumen de órgano que recibe la dosis **xx**. En este caso lo más habitual es que si no aparecen unidades se refiera a un valor de dosis en Gy. Por ejemplo un V20 inferior al 25% indicaría que se intenta conseguir que la dosis de 25 Gy sea recibida por menos del 25% del volumen del órgano afectado.

Las siguientes celdas muestran ejemplos de cómo *interrogar* a los objetos `dvh` sobre estos parámetros

In [ ]:
ptvboost_dvh.D5

In [ ]:
ptvboost_dvh.relative_volume.statistic('V62.45Gy')

In [ ]:
ptvboost_dvh.statistic('V62.45Gy')

### Cuestiones

9. Realizar una evaluación de los sigueientes criterios de prescripcioón de tratamiento y de tolerabilidad del tratamiento sobre el plan correspondiente


| Plan | Volumen | Criterio |
|----|-----|------|
|Mama | PTV mama dcha | Al menos 95% cubierto por el 95% de la dosis de prescripción (50 Gy) |
| | PTV mama dcha | No más del 5% cubierto por el 107% de la dosis de prescripción (50 Gy) |
| | PTV boost dcho | Al menos 95% cubierto por el 95% de la dosis de prescripción (50 Gy) |
| | PTV boost dcho | No más del 5% cubierto por el 107% de la dosis de prescripción (50 Gy) |
| | Pulmón | Volumen de órgano que recibe 20 Gy o más inferior al 25% |
|Boost | PTV boost dcho | Al menos 95% cubierto por el 95% de la dosis de prescripción (10 Gy) |
| | PTV boost dcho | No más del 5% cubierto por el 107% de la dosis de prescripción (10 Gy) |
|Suma | PTV boost dcho | Al menos 95% cubierto por el 95% de la dosis de prescripción (60 Gy) |
| | PTV boost dcho | No más del 5% cubierto por el 107% de la dosis de prescripción (60 Gy) |
| | Pulmón | Volumen de órgano que recibe 20 Gy o más inferior al 25% |
|Mama | PTV mama dcha | Al menos 95% cubierto por el 95% de la dosis de prescripción (40 Gy) |
| | PTV mama dcha | No más del 5% cubierto por el 107% de la dosis de prescripción (40 Gy) |
| | PTV boost dcho | Al menos 95% cubierto por el 95% de la dosis de prescripción (48 Gy) |
| | PTV boost dcho | No más del 5% cubierto por el 107% de la dosis de prescripción (48 Gy) |
| | Pulmón | Volumen de órgano que recibe 20 Gy o más inferior al 25% |


# Radiobiología
## Fraccionamiento y equivalencia de tratamientos

Segun el modelo $\alpha/\beta$ una dosis absorbida $D$ administrada en un número de fracciones $n$ produce una dosis biológica equivalente $DBE$ dada por
\begin{equation}
DBE = D\,(1 + \frac{D/n}{\alpha/\beta}) 
\end{equation}

donde el cociente $\alpha/\beta$ [Gy] es un parámetro específico del tejido que recibe la dosis.

Dos esquemas de tratamiento son equivalentes si producen la misma $DBE$.

### Cuestiones

10. Calcular qué dosis se tiene que administrar al volumen PTV boost dcho si se quiere realizar el tratamiento en lugar de en forma secuencial, 25+5 fracciones, en forma concurrente, 25 fracciones.

1. Repetir el cálculo anterior si queremos acelerar el tratamiento a 15 fracciones.

1. Calcular para el tratamiento concurrente en 15 fracciones cuál es la nueva restricción que se tiene que imponer sobre el pulmón para tener la misma probablidad de complicaciones que aceptamos en el tratamiento secuencial.

Considerar un $\alpha/\beta = 3$ tanto para el tejido de la mama como para el pulmón.

## Comparación de planes mediante DVHs

Los histogramas dosis volumen se pueden almacenar en bruto en archivos de datos (csv, excel, etc). En investigación a menudo se analizan colecciones de DVHs para obtener conclusiones.

En esta parte del cuaderno vamos a trabajar con los DVHs del tratamiento integrado y de la suma de las partes del tratamiento secuencial para analizar su equivalencia.

Ejecutando la siguiente celda descargamos el archivo `DVHs.xlsx` en formato Excel con los dos DVHs en la carpeta `data`

El archivo Excel tiene una hoja por cada tipo de tratamiento nombradas por la dosis media que debería recibir el PTV boost dcho como 48 Gy y 60 Gy respectivamente.

Los datos están en columnas con encabezados descriptivos:
- **Dr**: Dosis relativa a la dosis media del ptvBoost
- **D**: Dosis absoluta
- **Boost**: Dosis en el PTV boost dcho
- **Mama**: Dosis en el PTV mama dcha
- **Pulmon**: Dosis en en Pulmon Dcho

In [ ]:
# Descargar los datos compartidos de Google Drive
url = 'https://docs.google.com/spreadsheets/d/1GHyqh1MGdUisjAH0kltUAtf9bktWhaM9/edit?usp=sharing&ouid=109366532745677997406&rtpof=true&sd=true'
output = '../data/DVHs.xlsx'
gdown.download(url, output, quiet=False, fuzzy=True);

En python una buena forma de manejar datos en formato Excel es mediante el módulo `pandas`.

Este módulo lee la hoja indicada por `sheet_name` y almance los datos en un objeto de tipo `DataFrame` que dispone de muchos métodos para manipular y viualizar sus datos.

Para hacer gráficos el módulo de referencia es `matplotlib.pyplot`

Importamos `pandas` y `matplotlib.pyplot`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Leemos los datos de los histogramas en sendos DataFrames

In [ ]:
dvhSumadf = pd.read_excel('../data/DVHs.xlsx', sheet_name='60Gy')
dvhSIBdf = pd.read_excel('../data/DVHs.xlsx', sheet_name='48Gy')

Representamos los histogramas. Cada curva la podemos representar mediante el método `plot.line`  del correspondiente DataFrame. 

Para que todas las curvas aparezcan en la misma gráfica necesitamos un objeto `axis` que creamos mediante el método `subplots` de `matplollib.pyplot`

In [ ]:
fig, ax = plt.subplots()

dvhSumadf.plot.line(x='D', y='Boost', color='r', ax=ax, label='Suma Boost')
dvhSumadf.plot.line(x='D', y='Mama', color='b', ax=ax, label ='Suma Mama')
dvhSumadf.plot.line(x='D', y='Pulmon', color='g', ax=ax, label='Suma Pulmon')

dvhSIBdf.plot.line(x='D', y='Boost', color='m', ax=ax, linestyle='-', label='SIB Boost')
dvhSIBdf.plot.line(x='D', y='Mama', color='y', ax=ax, label='SIB Mama')
dvhSIBdf.plot.line(x='D', y='Pulmon', color='b', ax=ax, label='SIB Pulmon')

plt.legend(bbox_to_anchor=(1., 0.9))

ax.set_ylabel('% Volumen')
ax.set_xlabel('D [Gy]')

plt.show()

La figura anterior nos permite visualizar simultáneamente las dosis de dos tratamientos distintos, pero no son directamente comparables porque tienen fraccionamientos diferentes.

### Cuestiones

Añadir a cada DataFrame una columna con la **DBE** dosis biológica equivalente correspondiente a la dosis $D$. Represenetar gráficamentelos los histogramas en  una figura en función de la **DBE**

13. Analizar la equivalencia de los tratamientos expresados en **DBE**
1. Definiendo conformación de la dosis como el grado de solapamiento espacial entre la distribución de la dosis y los volúmenes de tratamiento, ¿qué esquema de tratamiento produce una mejor conformación? Razonar la respeusta comentando las diferencias en los DVHs de los tres volúmenes. 
1. Repetir la cuestión anterior pero en términos de la **DBEQ2**
1. Hacer una representación de los DVHs para el tratamiento secuencial en términos de la dosis $D$ y de **DBEQ2** y explicar porque no coinciden a pesar de que el concepto **DBEQ2** se introdujo para que los Oncólogos de Radioterapia pudieran comparar tratamientos realizados en fraccionamiento estándar de 2 Gy, que es cómo se ha realizado el tratamiento Suma.

Definimos la Dosis Equivalente Biológica como una función de la Dosis, el número de fracciones y el coeficinte $\alpha/\beta$ 

In [ ]:
def DBE_f(D, n, α_β=3):
    '''
    Calcula la Dosis Biológica Equivalente
    Argumentos:
    D: Dosis absorbida [Gy]
    n: Número de fracciones
    α_β: Razón α/β del volumen considerado [Gy]
    '''  
    d = D/n
    DBE = D * (1 + d/α_β)
    return DBE

def DBEQ2_f(D, n, α_β=3):
    '''
    Calcula la Dosis Biológica Equivalente en fraccines de 2 Gy
    Argumentos:
    D: Dosis absorbida [Gy]
    n: Número de fracciones
    α_β: Razón α/β del volumen considerado [Gy]
    '''  
    d = D/n
    DBEQ2 = D * (1 + d/α_β)/(1 + 2/α_β)
    return DBEQ2

In [ ]:
dvhSumadf['DBE'] = DBE_f(dvhSumadf.D, 30)
dvhSIBdf['DBE'] = DBE_f(dvhSIBdf.D, 15)
dvhSumadf['DBEQ2'] = DBEQ2_f(dvhSumadf.D, 30)
dvhSIBdf['DBEQ2'] = DBEQ2_f(dvhSIBdf.D, 15)

In [ ]:
fig, ax = plt.subplots()

dvhSumadf.plot.line(x='DBE', y='Boost', color='r', ax=ax, label='Suma Boost')
dvhSumadf.plot.line(x='DBE', y='Mama', color='b', ax=ax, label ='Suma Mama')
dvhSumadf.plot.line(x='DBE', y='Pulmon', color='g', ax=ax, label='Suma Pulmon')

dvhSIBdf.plot.line(x='DBE', y='Boost', color='m', ax=ax, linestyle='-', label='SIB Boost')
dvhSIBdf.plot.line(x='DBE', y='Mama', color='y', ax=ax, label='SIB Mama')
dvhSIBdf.plot.line(x='DBE', y='Pulmon', color='b', ax=ax, label='SIB Pulmon')

plt.legend(bbox_to_anchor=(1., 0.9))

ax.set_ylabel('% Volumen')
ax.set_xlabel('DBE [Gy]')

plt.show()

In [ ]:
fig, ax = plt.subplots()

dvhSumadf.plot.line(x='DBEQ2', y='Boost', color='r', ax=ax, label='Suma Boost')
dvhSumadf.plot.line(x='DBEQ2', y='Mama', color='b', ax=ax, label ='Suma Mama')
dvhSumadf.plot.line(x='DBEQ2', y='Pulmon', color='g', ax=ax, label='Suma Pulmon')

dvhSIBdf.plot.line(x='DBEQ2', y='Boost', color='m', ax=ax, linestyle='-', label='SIB Boost')
dvhSIBdf.plot.line(x='DBEQ2', y='Mama', color='y', ax=ax, label='SIB Mama')
dvhSIBdf.plot.line(x='DBEQ2', y='Pulmon', color='b', ax=ax, label='SIB Pulmon')

plt.legend(bbox_to_anchor=(1., 0.9))

ax.set_ylabel('% Volumen')
ax.set_xlabel('DBEQ2 [Gy]')

plt.show()

In [ ]:
fig, ax = plt.subplots()

dvhSumadf.plot.line(x='D', y='Boost', color='r', ax=ax, label='Suma Boost vs D')
dvhSumadf.plot.line(x='D', y='Mama', color='b', ax=ax, label ='Suma Mama vs D')
dvhSumadf.plot.line(x='D', y='Pulmon', color='g', ax=ax, label='Suma Pulmon vs D')

dvhSumadf.plot.line(x='DBEQ2', y='Boost', color='m', ax=ax, label='Suma Boost vs DBEQ2')
dvhSumadf.plot.line(x='DBEQ2', y='Mama', color='y', ax=ax, label ='Suma Mama vs DBEQ2')
dvhSumadf.plot.line(x='DBEQ2', y='Pulmon', color='b', ax=ax, label='Suma Pulmon vs DBEQ2')

plt.legend(bbox_to_anchor=(1., 0.9))

ax.set_ylabel('% Volumen')
ax.set_xlabel('D, DBEQ2 [Gy]')

plt.show()

### Cuestión

17. Estudiar qué efecto tiene el fraccionamiento sobre la tolerabilidad del tratamiento si el α/β del pulmón fuera diferente al de la mama y al del tejido tumoral. Considerar un rango de variación α/β ∈ (1, 10)

Calculamos la dosis biológica equivalente para los valores extremos del intervalo de variación del coeficiente $\alpha/\beta$

In [ ]:
dvhSIBdf['DBE_αβ1'] = DBE_f(dvhSumadf.D, 30, 1)
dvhSIBdf['DBE_αβ10'] = DBE_f(dvhSIBdf.D, 15, 10)

Representamos los histogramas

In [ ]:
fig, ax = plt.subplots()

dvhSIBdf.plot.line(x='DBE', y='Boost', color='r', ax=ax, label='SIB Boost α/β=3')
dvhSIBdf.plot.line(x='DBE', y='Mama', color='b', ax=ax, label ='SIB Mama α/β=3')
dvhSIBdf.plot.line(x='DBE', y='Pulmon', color='g', ax=ax, label='SIB Pulmon α/β=3')

dvhSIBdf.plot.line(x='DBE_αβ1', y='Pulmon', color='m', ax=ax, linestyle='-', label='SIB Pulmon α/β=1')
dvhSIBdf.plot.line(x='DBE_αβ10', y='Pulmon', color='y', ax=ax, label='SIB Pulmon α/β=10')

plt.legend(bbox_to_anchor=(1., 0.9))

ax.set_ylabel('% Volumen')
ax.set_xlabel('DBE [Gy]')

plt.show()

Se observa que si el cociente α/β es inferior para los volúmenes de tratamiento que para los órganos de riesgo el hipofraccionamiento mejora la tolerabilidad del tratamiento. Esta
es la situación que se da en el caso del cáncer de próstata que se cree que tiene un α/β de 2 y en el que se llegan a realizar tratamientos en tres fracciones que de manera convencional se realizaban en 39 fracciones. Para que estos tratamientos sean realizables en la práctica es necesario utilizar técnicas sofisticadas que aseguren que la dosis se administra con suficienrte precisión.
En cambio si el α/β es inferior para los órganos de riesgo que para los volúmenes de tratamiento, la tolerabilidad se reduce y es posible que no seamos capaces de cumplir
con las restricciones del tratamiento. En este caso para poder hipofraccionar se requiere
aumentar la exigencia sobre el diseño del tratamiento.

## Propuestas *premium*

Cuestiones voluntarias, desarrollos interesantes que requieren tener conocimientos básicos de python

1. Desarollar un gráfico que permita visualizar la distribución de dosis de forma interactiva y en en el se pueda seleccionear el intervalo de dosis $(D_{min}, D_{max})$, es decir, un gráfico en el se se visualizan por ejemplo las dosis comprendidas entre el 95% y el 107% de la dosis de prescripción.
1. Desarrollar un gráfico en el que se represente la dosis biológica equivalente de los distintos esquemas de tratamiento. 